# Mata Kuliah Bisnis Cerdas
- Dr. Eng. Farrikh Alzami, M.Kom
- Program Studi Sistem Informasi Fakultas Ilmu Komputer Universitas Dian Nuswantoro
- implementasi Gurobi untuk model building in mathematical programming

# Model Optimasi Pabrik Keripik Tradisional - Factory Planning

## Tujuan dan Prasyarat

Model ini bertujuan untuk mengoptimalkan produksi keripik tradisional dengan memaksimalkan profit sambil mempertimbangkan berbagai batasan operasional. Model ini menggunakan pemrograman linear untuk menentukan rencana produksi optimal.

Untuk menggunakan model ini, Anda perlu memahami:
- Dasar pemrograman Python
- Konsep dasar optimasi
- Penggunaan Gurobi Optimizer
- Pemahaman tentang pemrograman linier

## Deskripsi Masalah

Sebuah pabrik keripik tradisional memproduksi empat jenis produk:
- Keripik Singkong
- Keripik Pisang
- Keripik Ubi
- Keripik Kentang

Menggunakan tiga jenis mesin:
- 2 Mesin Pengupas
- 3 Mesin Penggorengan
- 2 Mesin Pengemas

Setiap produk memiliki:
- Profit kontribusi per unit (harga jual dikurangi biaya bahan baku)
- Waktu proses yang dibutuhkan pada setiap mesin
- Batasan bahan baku dari supplier lokal
- Faktor konversi dari bahan mentah ke produk jadi

## Formulasi Model Matematis

### Set dan Indeks

$$t \in \text{Bulan} = \{\text{Jan}, \text{Feb}, \text{Mar}, \text{Apr}\}$$: Set periode waktu

$$p \in \text{Produk} = \{\text{Singkong}, \text{Pisang}, \text{Ubi}, \text{Kentang}\}$$: Set produk

$$m \in \text{Mesin} = \{\text{Pengupas}, \text{Penggoreng}, \text{Pengemas}\}$$: Set mesin

### Parameter

$ \text{jam\_per\_bulan} \in \mathbb{R}^+ $: Waktu tersedia per bulan (jam)
- 2 shift × 8 jam × 24 hari = 384 jam

$\text{max\_inventory} \in \mathbb{N}$: Kapasitas penyimpanan maksimum (kg)
- 200 kg per produk

$\text{biaya\_simpan} \in \mathbb{R}^+$: Biaya penyimpanan per unit per bulan
- Rp 2.000/kg/bulan

$\text{target\_simpan} \in \mathbb{N}$: Target inventory akhir periode
- 50 kg per produk

$\text{profit}_p \in \mathbb{R}^+$: Profit per unit produk $p$

$\text{terpasang}_m \in \mathbb{N}$: Jumlah mesin tipe $m$ yang terpasang

$\text{maintenance}_{t,m} \in \mathbb{N}$: Jumlah mesin tipe $m$ dalam maintenance pada bulan $t$

$\text{waktu\_proses}_{m,p} \in \mathbb{R}^+$: Waktu proses (jam) yang dibutuhkan mesin $m$ untuk produk $p$

$\text{max\_jual}_{t,p} \in \mathbb{N}$: Batas maksimum penjualan produk $p$ pada bulan $t$

$\text{bahan\_baku}_{t,p} \in \mathbb{N}$: Ketersediaan bahan baku untuk produk $p$ pada bulan $t$

$\text{konversi}_p \in \mathbb{R}^+$: Faktor konversi bahan mentah ke produk jadi untuk produk $p$

### Variabel Keputusan

$\text{produksi}_{t,p} \in \mathbb{R}^+$: Jumlah produk $p$ yang diproduksi pada bulan $t$

$\text{simpan}_{t,p} \in [0, \text{max\_inventory}]$: Jumlah produk $p$ yang disimpan pada akhir bulan $t$

$\text{jual}_{t,p} \in [0, \text{max\_jual}_{t,p}]$: Jumlah produk $p$ yang dijual pada bulan $t$

### Fungsi Tujuan

Maksimasi total profit:

$$\text{Maksimasi} \quad Z = \sum_{t \in \text{Bulan}}\sum_{p \in \text{Produk}}(\text{profit}_p \times \text{jual}_{t,p} - \text{biaya\_simpan} \times \text{simpan}_{t,p})$$

### Batasan

1. **Balance Awal (Januari)**:
   
   $$\text{produksi}_{\text{Jan},p} = \text{jual}_{\text{Jan},p} + \text{simpan}_{\text{Jan},p} \quad \forall p \in \text{Produk}$$

2. **Balance Bulanan**:
   
   $$\text{simpan}_{t-1,p} + \text{produksi}_{t,p} = \text{jual}_{t,p} + \text{simpan}_{t,p}$$
   $$\forall p \in \text{Produk}, t \in \text{Bulan} \setminus \{\text{Jan}\}$$

3. **Target Inventory Akhir**:
   
   $$\text{simpan}_{\text{Apr},p} = \text{target\_simpan} \quad \forall p \in \text{Produk}$$

4. **Kapasitas Mesin**:
   
   $$\sum_{p \in \text{Produk}}\text{waktu\_proses}_{m,p} \times \text{produksi}_{t,p} \leq \text{jam\_per\_bulan} \times (\text{terpasang}_m - \text{maintenance}_{t,m})$$
   $$\forall m \in \text{Mesin}, t \in \text{Bulan}$$

5. **Ketersediaan Bahan Baku**:
   
   $$\text{produksi}_{t,p} / \text{konversi}_p \leq \text{bahan\_baku}_{t,p}$$
   $$\forall t \in \text{Bulan}, p \in \text{Produk}$$

## Implementasi Python

Model ini diimplementasikan menggunakan Gurobi Python API dengan:
1. Definisi sets dan parameter
2. Inisialisasi model dan variabel keputusan
3. Penambahan batasan
4. Optimasi model
5. Analisis hasil

## Analisis Hasil

Hasil optimasi akan memberikan:
1. Rencana produksi optimal per bulan per produk
2. Rencana penjualan optimal per bulan per produk
3. Rencana inventory optimal per bulan per produk
4. Total profit maksimal yang dapat dicapai

## Catatan Tambahan

- Model mengasumsikan produksi dapat dalam bentuk pecahan
- Biaya penyimpanan sama untuk semua jenis produk
- Faktor konversi bahan mentah ke produk jadi bersifat linear
- Tidak ada sequence-dependent setup times
- Jam kerja terdiri dari 2 shift per hari

In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 84.4 MB/s eta 0:00:00


In [4]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

# Definisi parameter dasar
products = ["Singkong", "Pisang", "Ubi", "Kentang"]
machines = ["Pengupas", "Penggoreng", "Pengemas"]
months = ["Jan", "Feb", "Mar", "Apr"]

# Profit per unit (dalam ribuan rupiah)
profit = {
    "Singkong": 15,  # Rp 15.000/kg
    "Pisang": 20,    # Rp 20.000/kg
    "Ubi": 18,       # Rp 18.000/kg
    "Kentang": 25    # Rp 25.000/kg
}

# Waktu proses per unit (dalam jam)
time_req = {
    "Pengupas": {
        "Singkong": 0.3,
        "Pisang": 0.4,
        "Ubi": 0.35,
        "Kentang": 0.25
    },
    "Penggoreng": {
        "Singkong": 0.5,
        "Pisang": 0.4,
        "Ubi": 0.45,
        "Kentang": 0.5
    },
    "Pengemas": {
        "Singkong": 0.2,
        "Pisang": 0.2,
        "Ubi": 0.2,
        "Kentang": 0.2
    }
}

# Jumlah mesin tersedia
installed = {
    "Pengupas": 2,
    "Penggoreng": 3,
    "Pengemas": 2
}

# Jadwal maintenance mesin (mesin tidak tersedia)
down = {
    ("Jan", "Pengupas"): 1,
    ("Feb", "Penggoreng"): 1,
    ("Mar", "Pengemas"): 1,
    ("Apr", "Penggoreng"): 1
}

# Batasan penjualan maksimum per bulan (dalam kg)
max_sales = {
    ("Jan", "Singkong"): 800,
    ("Jan", "Pisang"): 600,
    ("Jan", "Ubi"): 500,
    ("Jan", "Kentang"): 400,
    ("Feb", "Singkong"): 1000,
    ("Feb", "Pisang"): 700,
    ("Feb", "Ubi"): 600,
    ("Feb", "Kentang"): 500,
    ("Mar", "Singkong"): 900,
    ("Mar", "Pisang"): 800,
    ("Mar", "Ubi"): 700,
    ("Mar", "Kentang"): 600,
    ("Apr", "Singkong"): 1200,
    ("Apr", "Pisang"): 900,
    ("Apr", "Ubi"): 800,
    ("Apr", "Kentang"): 700
}

# Parameter inventory
holding_cost = 2      # Biaya penyimpanan Rp 2.000/kg/bulan
max_inventory = 200   # Kapasitas gudang maksimum 200 kg per produk
store_target = 50     # Target inventory akhir 50 kg per produk
hours_per_month = 2 * 8 * 24  # 2 shift, 8 jam, 24 hari kerja

# Batasan ketersediaan bahan baku lokal (dalam kg)
raw_material_supply = {
    ("Jan", "Singkong"): 1000,  # Ketersediaan singkong lokal
    ("Jan", "Pisang"): 800,     # Ketersediaan pisang lokal
    ("Jan", "Ubi"): 700,        # Ketersediaan ubi lokal
    ("Jan", "Kentang"): 500,    # Ketersediaan kentang lokal
    ("Feb", "Singkong"): 1200,
    ("Feb", "Pisang"): 900,
    ("Feb", "Ubi"): 800,
    ("Feb", "Kentang"): 600,
    ("Mar", "Singkong"): 1100,
    ("Mar", "Pisang"): 1000,
    ("Mar", "Ubi"): 900,
    ("Mar", "Kentang"): 700,
    ("Apr", "Singkong"): 1400,
    ("Apr", "Pisang"): 1100,
    ("Apr", "Ubi"): 1000,
    ("Apr", "Kentang"): 800
}

# Konversi bahan baku ke produk jadi (kg bahan baku -> kg produk jadi)
conversion_rate = {
    "Singkong": 0.4,  # 1 kg singkong mentah -> 0.4 kg keripik
    "Pisang": 0.3,    # 1 kg pisang mentah -> 0.3 kg keripik
    "Ubi": 0.4,       # 1 kg ubi mentah -> 0.4 kg keripik
    "Kentang": 0.35   # 1 kg kentang mentah -> 0.35 kg keripik
}

# Inisialisasi model
model = gp.Model('Pabrik Keripik')

# Variabel Keputusan
make = model.addVars(months, products, name="Make")  # jumlah produksi
store = model.addVars(months, products, ub=max_inventory, name="Store")  # jumlah simpan
sell = model.addVars(months, products, ub=max_sales, name="Sell")  # jumlah jual

# Fungsi Tujuan: Maksimalkan profit
obj = gp.quicksum(profit[product] * sell[month, product] - holding_cost * store[month, product]
                 for month in months for product in products)
model.setObjective(obj, GRB.MAXIMIZE)

# Batasan 1: Balance awal (Januari)
model.addConstrs(
    (make[months[0], product] == sell[months[0], product] + store[months[0], product]
     for product in products), name="Initial_Balance"
)

# Batasan 2: Balance bulanan
model.addConstrs(
    (store[months[months.index(month)-1], product] + make[month, product] ==
     sell[month, product] + store[month, product]
     for product in products for month in months if month != months[0]),
    name="Balance"
)

# Batasan 3: Target inventory akhir
model.addConstrs(
    (store[months[-1], product] == store_target for product in products),
    name="End_Balance"
)

# Batasan 4: Kapasitas mesin
model.addConstrs(
    (gp.quicksum(time_req[machine][product] * make[month, product]
                for product in products)
     <= hours_per_month * (installed[machine] - down.get((month, machine), 0))
     for machine in machines for month in months),
    name="Machine_Capacity"
)

# Batasan 5: Ketersediaan bahan baku lokal
model.addConstrs(
    (make[month, product] / conversion_rate[product] <= raw_material_supply[month, product]
     for month in months for product in products),
    name="Raw_Material"
)

# Optimasi model
model.optimize()

# Fungsi untuk menampilkan hasil dalam format DataFrame
def display_results(var_dict, name):
    rows = months.copy()
    columns = products.copy()
    result_df = pd.DataFrame(columns=columns, index=rows, data=0.0)

    for (month, product) in var_dict.keys():
        if abs(var_dict[month, product].x) > 1e-6:
            result_df.loc[month, product] = np.round(var_dict[month, product].x, 1)

    print(f"\n{name}:")
    return result_df

# Tampilkan hasil jika optimal
if model.status == GRB.OPTIMAL:
    print(f"\nProfit Optimal: Rp {model.objVal:,.2f}")

    # Tampilkan rencana produksi
    print("\nRencana Produksi (kg):")
    print(display_results(make, "Production Plan"))

    # Tampilkan rencana penjualan
    print("\nRencana Penjualan (kg):")
    print(display_results(sell, "Sales Plan"))

    # Tampilkan rencana inventory
    print("\nRencana Inventory (kg):")
    print(display_results(store, "Inventory Plan"))

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 48 rows, 48 columns and 128 nonzeros
Model fingerprint: 0x629bf52e
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [2e+02, 1e+03]
  RHS range        [5e+01, 1e+03]
Presolve removed 48 rows and 48 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.3930000e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.393000000e+04

Profit Optimal: Rp 93,930.00

Rencana Produksi (kg):

Production Plan:
     Singkong  Pisang    Ubi  Kentang
Jan     400.0   240.0  280.0    175.0
Feb     480.0   270.0  320.0    210.0
Mar     

# Analisis Hasil

In [6]:
# Import required libraries
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np

def analyze_machine_utilization(model, make, time_req, machines, products, months, hours_per_month, installed, down):
    """Menganalisis utilisasi mesin"""
    utilization = pd.DataFrame(index=months, columns=machines)

    for month in months:
        for machine in machines:
            used_hours = sum(time_req[machine].get(product, 0) * make[month, product].x
                           for product in products)
            available_hours = hours_per_month * (installed[machine] - down.get((month, machine), 0))
            utilization.loc[month, machine] = (used_hours / available_hours) * 100

    return utilization.round(2)

def analyze_raw_material_usage(make, conversion_rate, raw_material_supply, months, products):
    """Menganalisis penggunaan bahan baku"""
    usage = pd.DataFrame(index=months, columns=products)

    for month in months:
        for product in products:
            production = make[month, product].x
            raw_material_used = production / conversion_rate[product]
            raw_material_available = raw_material_supply[month, product]
            usage_percentage = (raw_material_used / raw_material_available) * 100
            usage.loc[month, product] = usage_percentage

    return usage.round(2)

def analyze_financial_metrics(model, make, sell, store, profit, holding_cost, months, products):
    """Menganalisis metrik keuangan"""
    financial_metrics = {}

    # Total revenue per product
    revenue_by_product = {p: sum(profit[p] * sell[m, p].x for m in months)
                         for p in products}

    # Total holding cost per product
    holding_by_product = {p: sum(holding_cost * store[m, p].x for m in months)
                         for p in products}

    # Net profit per product
    net_profit_by_product = {p: revenue_by_product[p] - holding_by_product[p]
                            for p in products}

    financial_metrics['revenue'] = revenue_by_product
    financial_metrics['holding_cost'] = holding_by_product
    financial_metrics['net_profit'] = net_profit_by_product
    financial_metrics['total_profit'] = model.objVal

    return financial_metrics

# Jalankan analisis
if model.status == GRB.OPTIMAL:
    print("\n=== ANALISIS HASIL OPTIMASI PABRIK KERIPIK ===\n")

    # 1. Analisis Produksi dan Penjualan
    print("1. RENCANA PRODUKSI (kg):")
    production_plan = display_results(make, "Production Plan")
    print("\nTotal produksi per produk:")
    print(production_plan.sum().round(2))

    print("\n2. RENCANA PENJUALAN (kg):")
    sales_plan = display_results(sell, "Sales Plan")
    print("\nTotal penjualan per produk:")
    print(sales_plan.sum().round(2))

    print("\n3. RENCANA INVENTORY (kg):")
    inventory_plan = display_results(store, "Inventory Plan")

    # 4. Analisis Utilisasi Mesin
    print("\n4. UTILISASI MESIN (%):")
    machine_utilization = analyze_machine_utilization(
        model, make, time_req, machines, products, months,
        hours_per_month, installed, down
    )
    print(machine_utilization)

    # 5. Analisis Penggunaan Bahan Baku
    print("\n5. PENGGUNAAN BAHAN BAKU (% dari ketersediaan):")
    raw_material_usage = analyze_raw_material_usage(
        make, conversion_rate, raw_material_supply, months, products
    )
    print(raw_material_usage)

    # 6. Analisis Finansial
print("\n6. ANALISIS FINANSIAL:")
financial = analyze_financial_metrics(
    model, make, sell, store, profit, holding_cost, months, products
)

print("\nTotal Profit: Rp {:,.2f}".format(financial['total_profit']))
print("\nNet Profit per Produk:")
for product, profit_value in financial['net_profit'].items():
    # Perbaikan format string
    print(f"{product}: Rp {profit_value:,.2f}")

# 7. Kesimpulan dan Rekomendasi
print("\n7. KESIMPULAN DAN REKOMENDASI:")

# Analisis bottleneck
avg_utilization = machine_utilization.mean()
bottleneck_machine = avg_utilization.idxmax()
print(f"Bottleneck Machine: {bottleneck_machine} "
      f"(Rata-rata utilisasi: {avg_utilization[bottleneck_machine]:.2f}%)")

# Produk paling menguntungkan
most_profitable = max(financial['net_profit'].items(), key=lambda x: x[1])[0]
print(f"Produk Paling Menguntungkan: {most_profitable}")

# Analisis bahan baku kritis
raw_material_critical = raw_material_usage.mean().idxmax()
print(f"Bahan Baku Kritis: {raw_material_critical} "
      f"(Rata-rata penggunaan: {raw_material_usage.mean()[raw_material_critical]:.2f}%)")


=== ANALISIS HASIL OPTIMASI PABRIK KERIPIK ===

1. RENCANA PRODUKSI (kg):

Production Plan:

Total produksi per produk:
Singkong    1880.0
Pisang      1140.0
Ubi         1360.0
Kentang      910.0
dtype: float64

2. RENCANA PENJUALAN (kg):

Sales Plan:

Total penjualan per produk:
Singkong    1830.0
Pisang      1090.0
Ubi         1310.0
Kentang      860.0
dtype: float64

3. RENCANA INVENTORY (kg):

Inventory Plan:

4. UTILISASI MESIN (%):
      Pengupas Penggoreng   Pengemas
Jan  93.164062  44.227431  28.515625
Feb  54.231771  77.734375  33.333333
Mar   57.19401  54.210069  70.052083
Apr   66.40625    95.3125  40.885417

5. PENGGUNAAN BAHAN BAKU (% dari ketersediaan):
    Singkong Pisang    Ubi Kentang
Jan    100.0  100.0  100.0   100.0
Feb    100.0  100.0  100.0   100.0
Mar    100.0  100.0  100.0   100.0
Apr    100.0  100.0  100.0   100.0

6. ANALISIS FINANSIAL:

Total Profit: Rp 93,930.00

Net Profit per Produk:
Singkong: Rp 27,350.00
Pisang: Rp 21,700.00
Ubi: Rp 23,480.00
Kentang: R